In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Problem:**

**Na podstawie tekstu recenzji przewidzieć jej ocenę.**

Dane ze strony: https://www.kaggle.com/datasets/andrewmvd/trip-advisor-hotel-reviews
Tripadvisor Hotel Review Dataset file, from the publication:
Alam, M. H., Ryu, W.-J., Lee, S., 2016. Joint multi-grain topic senti- ment: modeling semantic aspects for online reviews. Information Sci- ences 339, 206–223.

## Wczytanie danych i wstępne sprawdzenie danych:

In [ ]:
import pandas as pd # importowanie biblioteki związanej z odczytem danych z pliku CSV

df = pd.read_csv('/content/drive/MyDrive/Projekt z NLP/tripadvisor_hotel_reviews.csv') # Wczytanie pliku csv

df.head()  #wyświetlam nagłówki i pierwsze 5 wierszy żeby zobaczyć jak wyglądają dane w pliku (co tam w ogóle jest)

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [ ]:
df.shape   #sprawdzam ile jest wierszy i kolumn w pliku

(20491, 2)

In [ ]:
df.info()   #Wyświetlam informacje o kolumnach: nazwa kolumny, typ, zliczenie danych niepustych (w ten prosty sposób sprawdzę czy brakuje jakichś danych).

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20491 entries, 0 to 20490
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  20491 non-null  object
 1   Rating  20491 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 320.3+ KB


In [ ]:
df.describe()   #Wyświetlam podsumowanie statystyczne dla danych z kolumny "Rating". Dzięki temu w prosty sposób sprawdzę czy faktycznie oceny są tylko w skali 1-5, czy gdzieś błędnie nie jest wpisane zero lub ocena wyższa niż 5.

,Rating
count,20491.000000
mean,3.952223
std,1.233030
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


Z wstępnej analizy danych wynika, że nie mam brakujących danych i nie mam błędnych danych (tzn. oceny są z zakresu od 1-5, nie ma np. jakiejś 11).

## Czyszczenie danych (zgodnie z zasadmi NLP):

In [ ]:
# Najpierw sprawdzam sobie jakie są stopwords w bibliotekach NLTK i spacy, potem na ich podstawie stworzę włane stopwords:

from spacy.lang.en.stop_words import STOP_WORDS #importuję stopwords z biblioteki spacy
import nltk
nltk.download('stopwords') #importuję zbiór danych stopwords z biblioteki NLTK

stopwords_spacy = list(STOP_WORDS)      #gotowa lista stop_words z biblioteki spacy
print("\n****************************************\n")
print(f"Stop words w tej liście spacy jest {len(stopwords_spacy)}, a wyglądają następująco:\n\n",stopwords_spacy)

stopwords_nltk = list(nltk.corpus.stopwords.words('english'))       #gotowa lista stop_words z biblioteki nltk
print("\n****************************************\n")
print(f"Stop words w tej liście nltk jest {len(stopwords_nltk)}, a wyglądają następująco:\n\n",stopwords_nltk)


****************************************

Stop words w tej liście spacy jest 326, a wyglądają następująco:

 ['of', 'part', 'still', 'whereupon', 'under', 'onto', 'then', 'that', 'thereafter', 'mostly', 'hereby', 'please', 'beside', 'therein', 'is', 'be', 'should', 'former', 'take', 'or', '‘ve', 'regarding', 'full', 'yet', 'they', 'elsewhere', 'after', 'among', 'just', 'much', 'moreover', 'namely', 'some', 'ca', 'anyone', 'say', 'top', 'fifteen', 'toward', 'even', 'between', 'he', 'except', 'an', 'whoever', 'beyond', 'because', 'thereby', 'thence', 'through', 'also', 'rather', 'him', 'three', 'everywhere', 'the', "'re", 'name', 'cannot', 'she', 'these', 'sometimes', 'which', 'become', 'your', 'everything', 'whatever', 'must', 'eleven', 'used', 'almost', 'whence', 'eight', 'enough', '’d', 'forty', 'seem', 'with', 'herself', 'six', 'about', 'across', 'towards', 'was', 'it', 'amount', 'mine', 'whither', 'ten', 'when', 'go', 'in', 'yours', 'meanwhile', 'really', 'seeming', 'besides', 'wer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Uważam, że do oceny recenzji będą potrzebne mi przeczenia. Z biblioteki spacy ze stopwords wybieram słowa, które nie są przeczeniami i tworzę własny stopwords

my_stopwords = ['th', 'see', 'whose', 'using', 'within', 'down', 'anyhow', 'seems', 'side', 'thereafter', 'five', 'in', 'become', 'take', 'because', 'upon', 'thus', 'four', 'while', '‘s', 'much', '’d', 'am', 'doing', 'whoever', 'hers', 'two', 'whereafter', 'was', 'his', 'under', 'seemed', 'besides', 'beside', "'d", 'at', 'again', 'thru', 'hereby', 'hereupon', 'or', 'wherein', 'twenty', 'former', '‘m', 'put', 'do', 'to', 'herein', 'off', 'used', 'whole', 'due', 'been', 'from', 'back', 'eleven', 'please', 'we', 'per', 'further', 'sometime', 'least', 'various', 'whether', 'three', 'us', 'else', 'too', 'wherever', 'same', 'other', 'first', 'own', 'one', 'were', 'them', 'anyway', 'however', 'along', 'why', 'of', 'afterwards', 'anyone', 'have', 'by', 'after', 'bottom', 'whereupon', 'where', 'this', 'between', 'really', 'may', 'well', 'sixty', 'could', 'several', 'around', 'myself', 'herself', 'up', 'already', 'quite', 'onto', 'becoming', 'name', 'him', 'more', '‘d', 'over', 'few', 'nine', 'keep', 'hereafter', 'some', 'yourself', 'serious', 'either', 'my', 'these', 'everyone', 'i', "'ve", 'unless', 'about', 'himself', 'latter', 'its', 'done', 'even', 'twelve', 'their', 'towards', 'might', 'although', 'always', 'throughout', 'for', 'her', 'did', 'below', '‘ve', 'there', 'seeming', 'that', 'only', 'our', 'whenever', '’m', 'those', 'ours', 'all', 'then', 'seem', 'therein', 'such', 'move', 'give', 'front', 'get', 'everything', 'others', 'whence', 'which', 'becomes', 'toward', 'as', 'whither', 'empty', "'m", 'though', 'another', 'the', 'what', 'they', 'show', 'once', 'somewhere', 'also', 'an', 'with', 'be', 'it', 'formerly', '’ll', 'during', 'go', 'so', 'otherwise', 'made', 'must', '’re', 'thence', 'beyond', 're', 'until', 'on', 'via', 'beforehand', 'ourselves', "'ll", 'a', 'sometimes', 'still', 'became', 'eight', 'last', 'make', 'would', 'meanwhile', 'next', 'yourselves', 'thereby', 'themselves', 'any', 'among', 'now', 'regarding', 'if', 'anything', 'less', 'out', 'will', 'are', 'six', 'everywhere', 'alone', 'across', 'amount', 'therefore', 'whatever', 'she', 'you', 'enough', 'through', 'behind', 'n’t', 'amongst', '’ve', 'often', 'somehow', 'forty', 'had', 'hundred', 'before', 'full', "'re", 'than', 'thereupon', 'each', 'and', 'yours', 'can', 'elsewhere', 'but', 'mine', 'against', 'part', 'n‘t', 'whereas', 'itself', 'top', 'rather', '’s', 'both', 'is', 'just', 'he', 'very', 'anywhere', 'indeed', 'most', 'perhaps', 'namely', 'say', 'ten', 'does', 'many', 'being', 'has', 'whom', 'above', 'hence', 'me', 'ca', 'something', 'how', 'when', 'since', 'whereby', 'who', 'almost', 'should', 'fifty', 'into', 'nevertheless', '‘ll', 'yet', '‘re', 'someone', 'together', 'mostly', 'third', 'every', 'here', "'s", 'latterly', 'moreover', 'fifteen', 'your', 'call', 'ever']

In [ ]:
import re # import biblioteki re, czyli wyrażeń regularnych, które będą użyte do oczyszczenia tesktu
nltk.download('averaged_perceptron_tagger') #algorytm tagowania części mowy, przypisuje konkretne etykiety gramatyczne do słów w tekście, potrzebne do funkcji poniżej get_wordnet_pos

# Deklaracja i definicja funkcji, która będzie czyściła dane
def clean_text(text):
    temp = text.lower() # Cały tekst na małe litery, aby nie było różnic
    temp = re.sub('\d', '', temp) # Wszystkie cyfry '\d' zamieni na nic ''
    temp = re.sub('<[^>]*>', '', temp)  #usunięcie tagów HTML
    emojis = re.findall('(?::|;|=)(?:-)?(?:\)|\(|d|p)', temp)   #do zmiennej emojis przypisuję znaki odpowiadające emotikonom, d i p z małej bo wcześniej zmieniłam litery na małe
    temp = re.sub('[\W]+', ' ', temp) + ' '.join(emojis).replace('-', '')   #przemieszczam emotikony na koniec tekstu, usuwam myślnik z emotikon
    temp = re.sub('_', '', temp) # Tutaj usuwam _
    temp = temp.strip() # Usunięcie białych znaków na początku i końcu
    return temp # Zwracam oczyszczony tekst


#sprawdzam czy moja definicja funkcji czyści tekst tak jak sobie tego życzę - tzn. jeśli w tekście są emotikony to mają zostać (mogą pomóc w ocenie), pozostałe znaki będące znakami interpunkcyjnymi mają zostać usunięte
tekst = "He loves her, and she says that she doesn't love him :) !, so loving him is strange. She loved him once. She has 15 dogs instead, although she used to have only 1 dog. In my opinion, 1 dog is better than 15 dogs..."
s = clean_text(tekst)
print(s)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


he loves her and she says that she doesn t love him so loving him is strange she loved him once she has dogs instead although she used to have only dog in my opinion dog is better than dogs :)


In [ ]:
# Stemming:
from nltk.stem.porter import PorterStemmer # Importowanie biblioteki do Porter Stemmer
porter = PorterStemmer() # obiekt, który będzie wykorzystany do stemmingu w funkcji text_tokenizer

# Lemantyzacja:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tag import pos_tag    #potrzebne do funkcji poniżej: get_wordnet_pos

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):  #Definicja, która zostanie zastosowana do lemantyzacji (chodzi o to, żeby lematizer nie uważał każdego słowa za rzeczownik co jest domyślne).
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV,
                "s": wordnet.ADJ_SAT}
    return tag_dict.get(tag, wordnet.NOUN)

# Deklaracja funkcji, która będzie przekazywana do vectorizatora -- w celu oczyszczenia tekstu
def text_tokenizer(text):
    text = clean_text(text) # wywołanie wcześniej przygotowanej funkcji do oczyszczenia
    words_after_lem = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in text.split()]   #splitem dzielę text na pojedyncze słowa oraz dokonuję lemantyzacji w celu uzyskania dokładnej formy podstawowej słowa, co pomoże w stemingu w następnym kroku (aby jak najbardziej unikać nieprawidłowych przekształceń)
    words_after_stem = [porter.stem(word) for word in words_after_lem]  #po lemantyzacji dokonuję jeszcze stemingu co pomoże w ujednoliceniu form słów i zmniejszeniu liczby słów do analizy
    return  [word for word in words_after_stem if word not in my_stopwords and len(word) > 2 or word == 'no' or word == 'ok']# do tego usuwam te, które są za krótkie lub są w stopwords, ale zostawiam wyrazy no oraz ok (bo mogą mieć znaczenie)

# Testowanie lemantyzacji ze stemingiem:
tekst = "ok :-) nothing special charge diamond member hilton decided chain shot 20th anniversary seattle, start booked suite paid extra website description not, suite bedroom bathroom standard hotel room, took printed reservation desk showed said things like tv couch ect desk clerk told oh mixed suites description kimpton website sorry free breakfast, got kidding, embassy suits sitting room bathroom bedroom unlike kimpton calls suite, 5 day stay offer correct false advertising, send kimpton preferred guest website email asking failure provide suite advertised website reservation description furnished hard copy reservation printout website desk manager duty did not reply solution, send email trip guest survey did not follow email mail, guess tell concerned guest.the staff ranged indifferent not helpful, asked desk good breakfast spots neighborhood hood told no hotels, gee best breakfast spots seattle 1/2 block away convenient hotel does not know exist, arrived late night 11 pm inside run bellman busy chating cell phone help bags.prior arrival emailed hotel inform 20th anniversary half really picky wanted make sure good, got nice email saying like deliver bottle champagne chocolate covered strawberries room arrival celebrate, told needed foam pillows, arrival no champagne strawberries no foam pillows great room view alley high rise building good not better housekeeping staff cleaner room property, impressed left morning shopping room got short trips 2 hours, beds comfortable.not good ac-heat control 4 x 4 inch screen bring green shine directly eyes night, light sensitive tape controls.this not 4 start hotel clean business hotel super high rates, better chain hotels seattle,  "
oczyszczony_tekst=text_tokenizer(tekst)
print("\n****************************************\n")
print("Klasyczny tekst, po podzieleniu:")
print(tekst.split())
print("Oczyszczony tekst")
print(oczyszczony_tekst)
print("\n****************************************\n")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...



****************************************

Klasyczny tekst, po podzieleniu:
['ok', ':-)', 'nothing', 'special', 'charge', 'diamond', 'member', 'hilton', 'decided', 'chain', 'shot', '20th', 'anniversary', 'seattle,', 'start', 'booked', 'suite', 'paid', 'extra', 'website', 'description', 'not,', 'suite', 'bedroom', 'bathroom', 'standard', 'hotel', 'room,', 'took', 'printed', 'reservation', 'desk', 'showed', 'said', 'things', 'like', 'tv', 'couch', 'ect', 'desk', 'clerk', 'told', 'oh', 'mixed', 'suites', 'description', 'kimpton', 'website', 'sorry', 'free', 'breakfast,', 'got', 'kidding,', 'embassy', 'suits', 'sitting', 'room', 'bathroom', 'bedroom', 'unlike', 'kimpton', 'calls', 'suite,', '5', 'day', 'stay', 'offer', 'correct', 'false', 'advertising,', 'send', 'kimpton', 'preferred', 'guest', 'website', 'email', 'asking', 'failure', 'provide', 'suite', 'advertised', 'website', 'reservation', 'description', 'furnished', 'hard', 'copy', 'reservation', 'printout', 'website', 'desk', 'manage

## Deklaracja i definicja funkcji, w której będziemy przetwarzać i dzielić dane

In [ ]:
from sklearn.model_selection import train_test_split

def split_and_vectorize_text(vectorizer, X, y, test_size = 0.3):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size)

  X_train_transform = vectorizer.fit_transform(X_train) #fit_transform koduje wartości.
  X_test_transform = vectorizer.transform(X_test)
  print('Wielkość danych po przetworzeniu:', X_train_transform.shape) 
  return X_train_transform, X_test_transform, y_train, y_test

## Ustawienie metody do wektoryzacji tekstu, zdefiniowanie obiektu do wektoryrazji tekstu, utworzenie zbiorów: uczącego i testowego.

In [ ]:
# Importujemy biblioteki z tym związane:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF
method = "TF-IDF" # wybieram metodę TF-IDF

test_size = 0.3 # Ustalam wielkość (procenty) zbioru uczącego

# Tworzenie wektoryzera, według ustalonej metody
vectorizer = TfidfVectorizer(tokenizer=text_tokenizer)

# Wywołujemy zdefiniowaną wcześniej funkcję
X_train, X_test, y_train, y_test = split_and_vectorize_text(vectorizer, df['Review'], df['Rating'], test_size)  # Tutaj już mamy atrybuty warunkowe do trenowania, testowania, atrybut decyzyjny dla zbioru trenującego i testowego.

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Wielkość danych po przetworzeniu: (14343, 29898)


## Uczenie maszynowe

In [ ]:
# Importowanie bibliotek do uczenia maszynowego i do oceny klasyfikacji
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix


# Wybrane klasyfikatory:
classifier = {
    'clf_SVC' : SVC(class_weight='balanced', break_ties=True),
    'clf_LR_mod' : LogisticRegression(fit_intercept=False, class_weight='balanced', solver = 'liblinear', max_iter=100000),
    'clf_LR' : LogisticRegression(max_iter=100000),
}

for name in classifier:
    classifier[name].fit(X_train, y_train)
    y_pred = classifier[name].predict(X_test) 

# Teraz przechodzę do walidacji mojego algorytmu (ocenę jego jakości).
    report = classification_report(y_test,y_pred)
    print("*"*10)
    print(name)
    print("*"*10)
    print(report)

    cm = confusion_matrix(y_test, y_pred)
    print(cm)

**********
clf_SVC
**********
              precision    recall  f1-score   support

           1       0.72      0.61      0.66       428
           2       0.45      0.49      0.47       518
           3       0.44      0.38      0.40       643
           4       0.53      0.61      0.56      1805
           5       0.78      0.73      0.76      2754

    accuracy                           0.63      6148
   macro avg       0.58      0.56      0.57      6148
weighted avg       0.64      0.63      0.63      6148

[[ 261  135   13    6   13]
 [  83  253  111   49   22]
 [  11  124  243  233   32]
 [   3   40  164 1094  504]
 [   2   12   27  689 2024]]
**********
clf_LR_mod
**********
              precision    recall  f1-score   support

           1       0.64      0.67      0.65       428
           2       0.42      0.45      0.43       518
           3       0.41      0.38      0.39       643
           4       0.54      0.47      0.50      1805
           5       0.74      0.80   

Uważam, że dla problemu przewidywania oceny, najważniejsza jest precyzja.

Najlepszymi klasyfikatoromi okazały się LogisticRegression (clf_LR) oraz SVC. Dokładność kolejno 62 i 63. Wyniki mają bardzo zbliżone. Obydwa klasyfikatory słabo radzą sobie z ocenami średnimi 2-4, natomiast dobrze radzą sobie z ocenami 1 i 5. Klasyfikator SVC lepiej radzi sobie z ocenami 1 i 5 (dla ocen 1 precyzja 72, dla ocen 5 precyzja 78), natomiast dla ocen 2-4 trochę lepiej radzi sobie klasyfikator LogisticRegression.

Gdyby zrobić skalę 3 - stopniową i oceny 1-2 przypisać do jednej oceny równej 1 (czyli niedostatecznie), ocenę 3 przypisać jako 2 (czyli dostatecznie), a oceny 4 i 5 do jednej oceny równej 3 (czyli bardzo dobry), wówczas najprawdopodobniej klasyfikacja dałaby bardzo dobre wyniki, ale chciałam zachować pierwotny system ocen (czyli skalę 1-5).